# Cargando un archivos de datos a un DataFrame

In [0]:
#importar módulos
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#crear sesión para poder acceder a todas las API de Spark
spark = SparkSession \
    .builder \
    .appName("Introducción a Spark DataFrame") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

#define el data schema del archivo que queremos leer
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])    

# lee el archivo csv (con el data schema definido) en un dataframe de Spark. Usa el delimitador "tab" 

purchaseDataframe = spark.read.csv(
    "/FileStore/tables/purchases-3.csv", 
    header=True, schema=purchaseSchema, sep="\t")

#Muestra 3 filas de nuestro Dataframe
purchaseDataframe.show(4)

+----------+----+----+----+-----+-------+
|      Date|Time|City|Item|Total|Payment|
+----------+----+----+----+-----+-------+
|2012-01-01|null|null|null| null|   null|
|2012-01-01|null|null|null| null|   null|
|2012-01-01|null|null|null| null|   null|
|2012-01-01|null|null|null| null|   null|
+----------+----+----+----+-----+-------+
only showing top 4 rows



## Contamos el número de filas, luego imprimimos el dataframe schema y las estadísticas de nuestra data.

In [0]:
#Cuenta el número de filas de nuestra dataframe
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)

#Muestra el dataframe schema
purchaseDataframe.printSchema()

#Muestra estadísticas del campo que elegimos "Total" 
purchaseDataframe.describe('Total').show()

## Creamos un nuevo dataframe como subconjunto del dataframe que cargamos inicialmente

In [0]:
#Crea un nuevo dataframe con las columnas "City" y "Total"
newDataframe = purchaseDataframe.select(purchaseDataframe['City'], 
                                              purchaseDataframe['Total'])

newDataframe.show(5); #Muestra las 3 primeras filas
newDataframe.printSchema() #Muestra el dataset schema del nuuevo dataframe

## Sumando un valor constante a todas las filas de una columna

In [0]:
#Sumando un valor constante de 10 a todas las filas de la columna "Total"
purchaseDataframe.select(purchaseDataframe['City'],purchaseDataframe['Total'],
                         purchaseDataframe['Total']+10).show(5)

## Uso de filtros en un dataframe usando una condición definida

In [0]:
#Filtra solo las filas cuyo valor de la columna "Total" sea mayor a 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(5)

## Ordenando un dataframe por una columna definida

In [0]:
#Ordena el dataframe por la columna "City"
sortedByCity = purchaseDataframe.orderBy('City').show(5)
sortedByCity = purchaseDataframe.orderBy('City', 'Total').show(100)

# Usando consultas SQL en un dataFrame

Crear un nuevo dataFrame a partir del subconjunto de nuestro dataFrame existente

In [0]:
#Vamos a crear una vista temporal a partir de nuestro dataframe inicial
purchaseDataframe.createOrReplaceTempView("purchaseSql")

#Selecciona las columnas "Total" y "Payment" de la vista recientemente creada
anotherNewDataframe = spark.sql("SELECT Total, Payment FROM purchaseSql")
anotherNewDataframe.show(5)

Ordenando los datos a partir de una columna definida

In [0]:
#Ordenando los datos por la columna "City"
orderByCity = spark.sql("SELECT * FROM purchaseSql ORDER BY City")
orderByCity.show(10)

Filtraremos la columna "Total" para valores mayores a 200 y lo ordenaremos por la columna "Payment"

In [0]:
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE Total>200 ORDER BY Payment")
filterAndSortWithSQL.show(10)